In [1]:
import chromadb
import pandas as pd
from pathlib import Path
from IPython.display import HTML
from sentence_transformers import SentenceTransformer

In [4]:
import sys
sys.path.append('../../src')
from DataLoader import DataLoader

## Vorbereitung

### Laden der vorbereitenten Vector-Datenbank

In [5]:
chroma_file = '/media/sz/Data/Connected_Lecturers/Opal_crosslab/processed/chroma_db'

chroma_client = chromadb.PersistentClient(path=chroma_file)
collection = chroma_client.get_or_create_collection(
            name="oer_connected_lecturer"
)

### Laden des AI generierten Datensatzes 

In [6]:
data_folder =  "/media/sz/Data/Connected_Lecturers/Opal_crosslab/raw"
data_ai = "OPAL_ai_meta.p"

df_ai = pd.read_pickle(Path(data_folder) / data_ai)
df_ai["filename"]=df_ai["pipe:ID"]+"."+df_ai["pipe:file_type"]
df_ai.drop(columns=["ai:keywords_gen","ai:keywords_dnb"],inplace=True)
df_ai.head(3)


,pipe:ID,pipe:file_type,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:dewey,filename
0,8I6sM5zapD60,pdf,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...",,8I6sM5zapD60.pdf
1,8ZICOHBmAHyQ,pdf,"(Leerstring, da kein Autor im Dokument erwähnt...",[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...",,8ZICOHBmAHyQ.pdf
2,8Lfz8SAKa6k0,pdf,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...",,8Lfz8SAKa6k0.pdf


In [7]:
files = "OPAL_files_attrib.p"

df_files = pd.read_pickle(Path(data_folder) / files)
df_files.drop(columns=["opal:comment", "opal:language", "opal:publicationMonth", "opal:publicationYear"],inplace=True)
df_files.head(3)

,opal:filename,opal:oer_permalink,opal:license,opal:author,opal:title,opal:revisedAuthor,pipe:ID,pipe:file_type
0,SC_Hardware.png,https://bildungsportal.sachsen.de/opal/oer/6TI...,CC BY 4.0 Int.,Daniela Dobeleit,Hardware zum Styroporschneiden,[Vorname='Daniela' Familienname='Dobeleit' Tit...,6TIlFGfTxXkw,png
1,beleg.pdf,https://bildungsportal.sachsen.de/opal/oer/8I6...,CC BY-SA 4.0 Int.,,,,8I6sM5zapD60,pdf
2,Bereich III.pdf,https://bildungsportal.sachsen.de/opal/oer/8ZI...,CC BY-NC-ND 4.0 Int.,,,,8ZICOHBmAHyQ,pdf


In [8]:
df_ai = df_ai.merge(df_files[['pipe:ID', "opal:oer_permalink"]], on="pipe:ID", how="left")
df_ai.drop(columns=["pipe:ID", "pipe:file_type"],inplace=True)
df_ai.head(3)

,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:dewey,filename,opal:oer_permalink
0,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...",,8I6sM5zapD60.pdf,https://bildungsportal.sachsen.de/opal/oer/8I6...
1,"(Leerstring, da kein Autor im Dokument erwähnt...",[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...",,8ZICOHBmAHyQ.pdf,https://bildungsportal.sachsen.de/opal/oer/8ZI...
2,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...",,8Lfz8SAKa6k0.pdf,https://bildungsportal.sachsen.de/opal/oer/8Lf...


## Suchanfragen

### Textbasiert in den Chunks der Datenbank

In [26]:
suchbegriff = "Transformator"

results = collection.get(where_document={"$contains": suchbegriff})


In [24]:
relevant = pd.DataFrame(results["metadatas"]).groupby("filename").agg({"page":"unique"})\
                                  .reset_index()\
                                  .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                                  .dropna(subset=["opal:oer_permalink"])\
                                  [["filename", "ai:title", "ai:revisedAuthor", "ai:affilation", "ai:dewey",
                                   "opal:oer_permalink","page"]]\
                                  .head(15)

HTML(relevant.to_html(render_links=True, escape=False))

,filename,ai:title,ai:revisedAuthor,ai:affilation,ai:dewey,opal:oer_permalink,page
0,10XV5OePU7suY.pdf,EMA-LAB-10XV5OePU7suY,[],,,https://bildungsportal.sachsen.de/opal/oer/10XV5OePU7suY,"[12, 31]"
1,10XqEI89gsBlI.pdf,EMA-LAB-131 - Versuchsprotokoll Drehstromasynchronmaschine mit Schleifringläufer,[],,,https://bildungsportal.sachsen.de/opal/oer/10XqEI89gsBlI,"[12, 31]"
2,11317Id_pG4R4.pdf,Fakultät Physik - Physikdidaktik Portfolio ZUM SEMINAR VERTIEFUNG DER PHYSIKDIDAKTIK,"[Vorname='Falk' Familienname='Beuchel' Titel='', Vorname='Kerstin' Familienname='Gedigk' Titel='', Vorname='Josef' Familienname='Leisen' Titel='']",Oberschule Rödertal,,https://bildungsportal.sachsen.de/opal/oer/11317Id_pG4R4,[22]
3,11DjIJ7kvckfs.pdf,Grundlagen Sft: 8. Werkst. Bauw. 8.3. Alu.-Legierungen,[],TU Berlin,,https://bildungsportal.sachsen.de/opal/oer/11DjIJ7kvckfs,[20]
4,11HCuNyCVZVoQ.pdf,2. Werkstoffe und Bauweisen,"[Vorname='Werner' Familienname='Steinke' Titel='', Vorname='Schraut' Familienname='Leutenegger' Titel='', Vorname='Ganzer' Familienname='' Titel='']",,,https://bildungsportal.sachsen.de/opal/oer/11HCuNyCVZVoQ,[12]
5,11KHAOaGHYJwM.pdf,2. Werkstoffe und Bauweisen,"[Vorname='' Familienname='Havel' Titel='Havel', Vorname='' Familienname='Ganzer' Titel='Ganzer', Vorname='Werner' Familienname='Steinke' Titel='Werner Steinke', Vorname='' Familienname='Schraut' Titel='Schraut', Vorname='' Familienname='Leutenegger' Titel='Leutenegger', Vorname='' Familienname='Hassel' Titel='Hassel', Vorname='' Familienname='Ehrenstein' Titel='Ehrenstein']",,,https://bildungsportal.sachsen.de/opal/oer/11KHAOaGHYJwM,[12]
6,12-z6JEPV0n80.pdf,2. Werkstoffe und Bauweisen,"[Vorname='Havel' Familienname='' Titel='', Vorname='Taschinger' Familienname='' Titel='', Vorname='Werner' Familienname='Steinke' Titel='', Vorname='Ganzer' Familienname='' Titel='', Vorname='Schraut' Familienname='' Titel='', Vorname='Leutenegger' Familienname='' Titel='']",,,https://bildungsportal.sachsen.de/opal/oer/12-z6JEPV0n80,[12]
7,12X_RPO6yOouo.pdf,Bremsen und Bremsysteme für Schienenfahrzeuge,[],,,https://bildungsportal.sachsen.de/opal/oer/12X_RPO6yOouo,[20]
8,13CHB1bBHXX_k.pdf,Fahrzeugtechnik - Antriebskräfte und Zugkraftübertragung bei Schienenfahrzeugen,[],,,https://bildungsportal.sachsen.de/opal/oer/13CHB1bBHXX_k,[51]
9,17eBg-8vq8z3w.pdf,EÜV Lernpaket 7 Ohmsch-Induktives Systemelement,[Vorname='Wolfgang' Familienname='Schufft' Titel='Prof. Dr.-Ing.'],Fakultät für Elektrotechnik und Informationstechnik,,https://bildungsportal.sachsen.de/opal/oer/17eBg-8vq8z3w,[1]


### Embeddings-Basiert

In [37]:
model = SentenceTransformer("jinaai/jina-embeddings-v2-base-de", trust_remote_code=False)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-de and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

In [38]:
document = """
   Transformator sind elektrische Maschinen, die elektrische Energie in mechanische Energie umwandeln. 
   """

embedding = model.encode(document)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [39]:
results = collection.query(
    query_embeddings=[embedding],  # Anstatt query_texts
    n_results=30
)

In [40]:
relevant_ = pd.DataFrame(results["metadatas"][0]).groupby("filename").agg({"page":"unique"})\
                                  .reset_index()\
                                  .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                                  .dropna(subset=["opal:oer_permalink"])\
                                  [["filename", "ai:title", "ai:revisedAuthor", "ai:affilation", "ai:dewey",
                                   "ai:dewey", "opal:oer_permalink","page"]]\
                                  .head(30)

HTML(relevant_.to_html(render_links=True, escape=False))

,filename,ai:title,ai:revisedAuthor,ai:affilation,ai:dewey,ai:dewey,opal:oer_permalink,page
0,10JW63ui75ahk.pdf,Gestalttheorie und kognitive Psychologie,"[Vorname='Wolfgang' Familienname='Köhler' Titel='', Vorname='Kurt' Familienname='Koffka' Titel='', Vorname='Max' Familienname='Wertheimer' Titel='', Vorname='Wolfgang' Familienname='Metzger' Titel='', Vorname='Kurt' Familienname='Lewin' Titel='', Vorname='Bousfield' Familienname='' Titel='', Vorname='Picasso' Familienname='' Titel='', Vorname='Zeigarnik' Familienname='' Titel='']",,,,https://bildungsportal.sachsen.de/opal/oer/10JW63ui75ahk,[114]
1,10JilXvS3EydU.pdf,Wirtschaftsmathematik II Hausaufgabe 1,"[Vorname='A.' Familienname='Voß-Böhme' Titel='Prof. Dr.', Vorname='K.' Familienname='Fischer' Titel='Dr.']",Hochschule für Technik und Wirtschaft Dresden,,,https://bildungsportal.sachsen.de/opal/oer/10JilXvS3EydU,[0]
2,10UZ15z9_nbqo.pdf,Basics of Bibliographic Cataloging and Indexing,[],,,,https://bildungsportal.sachsen.de/opal/oer/10UZ15z9_nbqo,[199]
3,10gxOszvxz3to.pdf,Mathematik als formales System.,[],,,,https://bildungsportal.sachsen.de/opal/oer/10gxOszvxz3to,[37]
4,10u2FLKt0ieZs.pdf,Assoziatives Lernen,[],,,,https://bildungsportal.sachsen.de/opal/oer/10u2FLKt0ieZs,[131]
5,1135XPBYaZ1Os.pdf,Bibliothek Forschung für welche Praxis?,"[Vorname='Petra' Familienname='Hauke' Titel='', Vorname='Andrea' Familienname='Kaufmann' Titel='', Vorname='Vivien' Familienname='Petras' Titel='']",Humboldt-Universität,,,https://bildungsportal.sachsen.de/opal/oer/1135XPBYaZ1Os,[1]
6,12bvyBIZsCpao.pdf,Wirtschaftsmathematik II Hausaufgabe 6,"[Vorname='A.' Familienname='Voß-Böhme' Titel='Prof. Dr.', Vorname='K.' Familienname='Fischer' Titel='Dr.']",Hochschule für Technik und Wirtschaft Dresden,,,https://bildungsportal.sachsen.de/opal/oer/12bvyBIZsCpao,[0]
7,12pHudk4KYim8.pdf,Rechtssprache im Althochdeutschen und ihre Erforschung,"[Vorname='Gerhard' Familienname='Kobler' Titel='', Vorname='Francois Louis' Familienname='Ganshof' Titel='', Vorname='Helmut' Familienname='Ibach' Titel='', Vorname='Elisabeth Karg-Gasterstädt' Familienname='Karg-Gasterstädt' Titel='', Vorname='Florus van der' Familienname='Rhee' Titel='', Vorname='Ingo' Familienname='Reiffenstein' Titel='', Vorname='Stefan' Familienname='Sonderegger' Titel='', Vorname='Dagmar Hüpper-Dröge' Familienname='Hüpper-Dröge' Titel='', Vorname='Karl Hyldgaard-Jensen' Familienname='Hyldgaard-Jensen' Titel='', Vorname='Philipp' Familienname='Heck' Titel='', Vorname='Arno' Familienname='Schirokauer' Titel='']",,,,https://bildungsportal.sachsen.de/opal/oer/12pHudk4KYim8,[4]
8,13qH5TONqJVHc.pdf,Stilblüten in der Wissenschaftssprache,"[Vorname='Helmut' Familienname='Weinreich' Titel='', Vorname='Harald' Familienname='Weinrich' Titel='', Vorname='Roger' Familienname='Willemsen' Titel='', Vorname='Rüdiger' Familienname='Safranski' Titel='', Vorname='Bruno' Familienname='Latour' Titel='', Vorname='Alfred' Familienname='Schütz' Titel='', Vorname='Giovanni' Familienname='Trapattoni' Titel='', Vorname='Rudi' Familienname='Völler' Titel='']",,,,https://bildungsportal.sachsen.de/opal/oer/13qH5TONqJVHc,[5]
9,14EtI6BsvpwcY.pdf,Als die alten Medien neu waren - Medienrevolution in der Geschichte,[Vorname='Michael' Familienname='Giesecke' Titel=''],,,,https://bildungsportal.sachsen.de/opal/oer/14EtI6BsvpwcY,[4]
